# Generation without kv_cache

![NO_KVCACHE](./image/without_kv_cache.png)

In [1]:
import torch
import torch.nn.functional as F
from transformers import LlamaModel, LlamaConfig, LlamaForCausalLM
import time

config = LlamaConfig(
    vocab_size=100,
    hidden_size=256,
    intermediate_size=512,
    num_hidden_layers=2,
    num_attention_heads=4,
    num_key_value_heads=4,
)
model = LlamaForCausalLM(config)  # 加载模型

X = torch.randint(0, 100, (1, 10))  # 创建数据、不使用tokenizer
print(X.shape)

idx = {}
idx['input_ids'] = X
for i in range(4):
    print(f"\nGeneration第{i}个时的输入{idx['input_ids'].shape}：")
    print(f"Generation第{i}个时的输入{idx['input_ids']}：")
    output = model(**idx)
    logits = output['logits'][:, -1, :]
    idx_next = torch.argmax(logits, dim=1)[0]
    time.sleep(1)

    idx['input_ids'] = torch.cat((idx['input_ids'], idx_next.unsqueeze(0).unsqueeze(1)), dim=-1)

torch.Size([1, 10])

Generation第0个时的输入torch.Size([1, 10])：

Generation第0个时的输入tensor([[51, 44, 69,  6, 26, 31, 40, 51, 11, 18]])：

Generation第1个时的输入torch.Size([1, 11])：

Generation第1个时的输入tensor([[51, 44, 69,  6, 26, 31, 40, 51, 11, 18, 23]])：

Generation第2个时的输入torch.Size([1, 12])：

Generation第2个时的输入tensor([[51, 44, 69,  6, 26, 31, 40, 51, 11, 18, 23, 82]])：

Generation第3个时的输入torch.Size([1, 13])：

Generation第3个时的输入tensor([[51, 44, 69,  6, 26, 31, 40, 51, 11, 18, 23, 82, 23]])：

In [8]:
model.model.causal_mask
model.model._update_causal_mask(model.model.causal_mask, torch.randn((,2048,2048)))

╭─────────────────────────────────────── Traceback (most recent call last) ───────────────────────────────────────╮
│ in <module>:2                                                                                                   │
│                                                                                                                 │
│   1 model.model.causal_mask                                                                                     │
│ ❱ 2 model.model._update_causal_mask(model.model.causal_mask, torch.randn((100,2048,2048)))                      │
│   3                                                                                                             │
│                                                                                                                 │
│ /Users/denghang/Miniconda3/envs/llm/lib/python3.9/site-packages/transformers/models/llama/modeling_llama.py:107 │
│ 5 in _update_causal_mask                                                                                        │
│                                                                                                                 │
│   1072 │   │   causal_mask = causal_mask.to(dtype=dtype, device=device)                                         │
│   1073 │   │   if attention_mask is not None and attention_mask.dim() == 2:                                     │
│   1074 │   │   │   mask_length = attention_mask.shape[-1]                                                       │
│ ❱ 1075 │   │   │   padding_mask = causal_mask[..., :mask_length].eq(0.0) * attention_mask[:, No                 │
│   1076 │   │   │   causal_mask[..., :mask_length] = causal_mask[..., :mask_length].masked_fill(                 │
│   1077 │   │   │   │   padding_mask, torch.finfo(dtype).min                                                     │
│   1078 │   │   │   )                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: The size of tensor a (100) must match the size of tensor b (2048) at non-singleton dimension 0

# Generation With KV-Cache

![KVCACHE](./image/with_kv_cache.png)

In [8]:
# author: xiaodongguaAIGC
# KV-Cache + Generation + decoder

import torch
import torch.nn.functional as F

D = 128  # single-head-dim
V = 64  # vocab_size


# Decoder-Only
class xiaodonggua_kv_cache(torch.nn.Module):
    def __init__(self, D, V):
        super().__init__()
        self.D = D
        self.V = V
        self.Embedding = torch.nn.Embedding(V, D)
        self.Wq = torch.nn.Linear(D, D)
        self.Wk = torch.nn.Linear(D, D)
        self.Wv = torch.nn.Linear(D, D)
        self.lm_head = torch.nn.Linear(D, V)  # LM_head
        self.cache_K = self.cache_V = None  # initial

    def forward(self, X):
        X = self.Embedding(X)
        Q, K, V = self.Wq(X), self.Wk(X), self.Wv(X)
        print(f"input_Q:{Q.shape}")
        print(f"input_K:{Q.shape}")
        print(f"input_V:{Q.shape}")

        # Easy KV_Cache
        if self.cache_K == None:
            self.cache_K = K
            self.cache_V = V
        else:
            self.cache_K = torch.cat((self.cache_K, K), dim=1)
            self.cache_V = torch.cat((self.cache_V, V), dim=1)
            K = self.cache_K
            V = self.cache_V

        print(f"cache_K:{self.cache_K.shape}")
        print(f"cache_V:{self.cache_K.shape}")

        # ignore proj/MLP/scaled/mask/multi-head
        attn = Q @ K.transpose(1, 2) @ V

        # output
        output = self.lm_head(attn)
        return output

In [17]:
model = xiaodonggua_kv_cache(D, V)  # 创建decode模型

# 创建数据、不使用tokenizer
X = torch.randint(0, 64, (1, 10))
print(X.shape)

for i in range(3):
    print(f"\nGeneration {i} step input_shape: {X.shape}：")
    output = model.forward(X)
    next_token = torch.argmax(F.softmax(output, dim=-1), -1)[:, -1]
    print(f'next_token预测:{next_token}')
    X = next_token.unsqueeze(0)

torch.Size([1, 10])

Generation 0 step input_shape: torch.Size([1, 10])：

input_Q:torch.Size([1, 10, 128])

input_K:torch.Size([1, 10, 128])

input_V:torch.Size([1, 10, 128])

...

input_K:torch.Size([1, 1, 128])

input_V:torch.Size([1, 1, 128])

cache_K:torch.Size([1, 12, 128])

cache_V:torch.Size([1, 12, 128])

next_token预测:tensor([62])